# Реализация простейшей RNN для классификации текстов

In [1]:
import pandas as pd

from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

from torch.optim import Adam

from tqdm import tqdm

import torch.nn as nn

from sklearn.metrics import classification_report


d:\first_sprint\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

# загрузка датасета
raw = pd.read_csv('yelp_reviews.csv')


texts = raw['text']
labels = raw['label']


print('Размер датасета:')
print(len(texts))


print('Первый отзыв из датасета:')
print(texts[0])


print('И его рейтинг:')
print(labels[0]) 

Размер датасета:
6500
Первый отзыв из датасета:
Worst sandwich on Earth.\nI'd rather eat a dead whore.\nPlease...never come here.
И его рейтинг:
0


In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

In [10]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_texts_tokenized = tokenizer(train_texts.tolist(), truncation=True)['input_ids']
val_texts_tokenized = tokenizer(val_texts.tolist(), truncation=True)['input_ids']

In [13]:
# создаём класс кастомного, наследуясь от класса Dataset из PyTorch

class YelpDataset(Dataset):
    # в конструкторе просто сохраняем тексты и классы
    def __init__(self, texts, labels, max_len=256):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len


    # возвращаем размер датасета (кол-во текстов)
    def __len__(self):
        return len(self.texts)
        
    def __getitem__(self, idx):
        # возвращаем текст и его класс
        # для текста ограничиваем длину
        # не делаем никаких доп. преобразований как padding и masking
        return {
            'text': torch.tensor(self.texts[idx][:self.max_len], dtype=torch.long),
            'label': torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        }

In [14]:
# кастомная функция collate_fn для формирования батчей
def collate_fn(batch):
    texts = [item['text'] for item in batch] # получите список текстов в батче
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long) # получите список классов в батче
    lengths = torch.tensor([len(text) for text in texts], dtype=torch.long) # посчитайте список длин текстов в батче 
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0) # реализуйте паддинг для текстов


    return {
        'input_ids': padded_texts, 
        'lengths': lengths, 
        'labels': labels
    }

In [15]:
# создаём датасеты
train_dataset = YelpDataset(texts=train_texts_tokenized, labels=train_labels)
val_dataset = YelpDataset(texts=val_texts_tokenized, labels=val_labels)


batch_size = 64


# создаём даталоадеры
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


print(f'Количество батчей в train_dataloader: {len(train_dataloader)}')
print(f'Количество батчей в val_dataloader: {len(val_dataloader)}')


print('Размерности батчей:')
for batch in train_dataloader:
    print('input_ids:', batch['input_ids'].shape)
    print('lengths:', batch['lengths'].shape)
    print('labels:', batch['labels'].shape)
    break 

Количество батчей в train_dataloader: 82
Количество батчей в val_dataloader: 21
Размерности батчей:
input_ids: torch.Size([64, 256])
lengths: torch.Size([64])
labels: torch.Size([64])


In [19]:
# класс модели
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) # напишите слой эмбеддинга с входной размерной vocab_size и выходной embedding_dim
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True) # напишите слой RNN
        self.fc = nn.Linear(hidden_size, output_size) # линейный слой для получения скоров классификации


    def forward(self, input_ids, lengths):
        embedded = self.embedding(input_ids)
        packed = nn.utils.rnn.pack_padded_sequence(
        embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        ) # "запакуйте" тензор embedded, используя pack_padded_sequence
        
        packed_output, hidden = self.rnn(packed)# посчитайте выход rnn
        output, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        # Используем последнее скрытое состояние для классификации
        last_hidden = hidden[-1]
        out = self.fc(last_hidden)# посчитайте скоры для классификации по последнему скрытому состоянию (hidden[-1])
        return out

In [20]:
# создаём модель, оптимизатор, объявляем функцию потерь
# Получаем размер словаря из токенизатора
vocab_size = tokenizer.vocab_size  # например, 30522 для BERT

# Объявляем модель
model = SimpleRNN(
    vocab_size=vocab_size,
    embedding_dim=128,       # размерность эмбеддинга
    hidden_size=128,         # размер скрытого состояния RNN
    output_size=5            # 5 классов (отзывов)
)

# Функция потерь
loss_fn = nn.CrossEntropyLoss()

# Оптимизатор
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 10  # число эпох
train_losses = []

for epoch in range(n_epochs):
    model.train()
    total_train_loss = 0.0

    for batch in tqdm(train_dataloader):
        # 1. Получаем данные из батча
        inputs = batch['input_ids']
        lengths = batch['lengths']
        labels = batch['labels']

        # 2. Обнуляем градиенты
        optimizer.zero_grad()

        # 3. Прямой проход
        outputs = model(inputs, lengths)

        # 4. Считаем лосс
        loss = loss_fn(outputs, labels)

        # 5. Назад (обратное распространение ошибки) # считаем градиенты 
        loss.backward()

        # 6. Шаг оптимизации
        optimizer.step()

        # 7. Сохраняем лосс
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)

    # ----------- Оценка на валидации -----------

    model.eval()
    total_val_loss = 0.0
    y_true, y_pred = [], []

    for batch in tqdm(val_dataloader):
        inputs = batch['input_ids']
        lengths = batch['lengths']
        labels = batch['labels']

        with torch.no_grad():
            outputs = model(inputs, lengths)
            loss = loss_fn(outputs, labels)
            total_val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.tolist())
            y_pred.extend(preds.tolist())

    avg_val_loss = total_val_loss / len(val_dataloader)

    print(f"\nEpoch {epoch + 1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")
    print("Метрики классификации (валидация):")
    print(classification_report(y_true, y_pred))


100%|██████████| 21/21 [00:01<00:00, 17.78it/s]



Epoch 1: Train Loss = 1.6115, Val Loss = 1.5960
Метрики классификации (валидация):
              precision    recall  f1-score   support

           0       0.36      0.05      0.09       238
           1       0.26      0.14      0.18       301
           2       0.23      0.73      0.35       283
           3       0.14      0.02      0.03       242
           4       0.34      0.28      0.31       236

    accuracy                           0.25      1300
   macro avg       0.27      0.24      0.19      1300
weighted avg       0.27      0.25      0.20      1300



100%|██████████| 21/21 [00:00<00:00, 22.31it/s]



Epoch 2: Train Loss = 1.5487, Val Loss = 1.5921
Метрики классификации (валидация):
              precision    recall  f1-score   support

           0       0.25      0.25      0.25       238
           1       0.28      0.14      0.18       301
           2       0.24      0.54      0.33       283
           3       0.28      0.25      0.26       242
           4       0.41      0.10      0.16       236

    accuracy                           0.26      1300
   macro avg       0.29      0.26      0.24      1300
weighted avg       0.29      0.26      0.24      1300



100%|██████████| 21/21 [00:01<00:00, 18.96it/s]


...


Epoch 8: Train Loss = 0.9462, Val Loss = 1.8750
Метрики классификации (валидация):
              precision    recall  f1-score   support

           0       0.33      0.40      0.36       238
           1       0.32      0.40      0.35       301
           2       0.20      0.17      0.19       283
           3       0.29      0.22      0.25       242
           4       0.31      0.27      0.29       236

    accuracy                           0.29      1300
   macro avg       0.29      0.29      0.29      1300
weighted avg       0.29      0.29      0.29      1300



100%|██████████| 21/21 [00:01<00:00, 19.09it/s]



Epoch 9: Train Loss = 0.8271, Val Loss = 2.0419
Метрики классификации (валидация):
              precision    recall  f1-score   support

           0       0.34      0.34      0.34       238
           1       0.33      0.21      0.26       301
           2       0.19      0.17      0.18       283
           3       0.25      0.42      0.31       242
           4       0.32      0.28      0.30       236

    accuracy                           0.28      1300
   macro avg       0.29      0.28      0.28      1300
weighted avg       0.29      0.28      0.27      1300



100%|██████████| 21/21 [00:01<00:00, 14.43it/s]


Epoch 10: Train Loss = 0.6995, Val Loss = 2.1736
Метрики классификации (валидация):
              precision    recall  f1-score   support

           0       0.33      0.39      0.36       238
           1       0.32      0.21      0.26       301
           2       0.21      0.22      0.21       283
           3       0.27      0.40      0.32       242
           4       0.32      0.22      0.26       236

    accuracy                           0.28      1300
   macro avg       0.29      0.29      0.28      1300
weighted avg       0.29      0.28      0.28      1300



## Резюме по задаче

В этой задаче мы реализовали простую рекуррентную нейронную сеть (SimpleRNN) для классификации отзывов из датасета Yelp. Целью было не построение идеальной модели, а отработка всех этапов работы с RNN: от предобработки текста до обучения модели с учётом длины последовательностей.

После запуска обучения видно, что начиная примерно с 5-й эпохи модель начинает переобучаться. Метрики на валидации остаются довольно низкими, но это ожидаемо для такой базовой архитектуры без регуляризации и подбора гиперпараметров.

Сама модель получилась простой, что и требовалось — как основа для понимания, как устроены RNN и как они применяются в задачах обработки текстов